In [8]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is the meaning of {word}?")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={
        "max_new_tokens": 250
    },
)

chain = prompt | llm

chain.invoke({
    "word": "potato"
})

ValidationError: 1 validation error for HuggingFaceHub
  Value error, Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. [type=value_error, input_value={'repo_id': 'mistralai/Mi...acehub_api_token': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error